# Creación de capas personalizadas

En ciertas ocasiones es posible que se desee implementar una nueva capa de la red neuronal artificial para la que no existe una implementación en Tensorflow o Keras. En estos casos podemos utilizar Tensorflow para crear capas personalizadas.

### Importando el conjunto de datos

In [12]:
# <-- analytics -->
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

# <-- machine learning -->
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from keras import datasets

# <-- deep learning -->
import tensorflow as tf
from keras import models
from keras import layers
import keras

In [13]:
boston_housing = datasets.boston_housing

In [14]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

In [15]:
features = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

df_train = pd.DataFrame(np.column_stack([X_train, y_train]), columns=features)
df_train.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,15.2
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,42.3
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,17.7
5,0.28392,0.0,7.38,0.0,0.493,5.708,74.3,4.7211,5.0,287.0,19.6,391.13,11.74,18.5
6,9.18702,0.0,18.10,0.0,0.700,5.536,100.0,1.5804,24.0,666.0,20.2,396.90,23.60,11.3
7,4.09740,0.0,19.58,0.0,0.871,5.468,100.0,1.4118,5.0,403.0,14.7,396.90,26.42,15.6
8,2.15505,0.0,19.58,0.0,0.871,5.628,100.0,1.5166,5.0,403.0,14.7,169.27,16.65,15.6
9,1.62864,0.0,21.89,0.0,0.624,5.019,100.0,1.4394,4.0,437.0,21.2,396.90,34.41,14.4


### División del conjunto de datos

In [16]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

### Escalando el conjunto de datos

In [17]:
scaler = RobustScaler()

X_train_prep = scaler.fit_transform(X_train)
X_val_prep = scaler.transform(X_val)
X_test_prep = scaler.transform(X_test)

### Definicion de la arquitectura de la Red Neuronal Artificial

### Creación de capas sin parámetros

Las capas sin parámetros aplican transformaciones sobre los valores de entrada. Algunos ejemplos de capas de este tipo que se encuentran implementados en Keras son: _keras.layers.Flatten_ o _keras.layers.ReLU_. 

En estos casos, la mejor forma de crear este tipo de capas es desarrollar una función personalizada y utilizarla junto con la construcción _keras.layers.Lambda_.

In [18]:
# Capa personalizada que eleva al cuadrado los valores de entrada
square_layer = keras.layers.Lambda(lambda x: tf.square(x))

Esta capa se utiliza de la misma forma que el resto de capas que se encuentran implementadas en Keras por defecto

In [19]:
network = models.Sequential()

network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(square_layer)
network.add(layers.Dense(1))

/Users/cristianariel/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741 (2.89 KB)

 Trainable params: 741 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
network.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae'])

In [22]:
history = network.fit(X_train_prep, 
                      y_train, 
                      epochs=50, 
                      validation_data=(X_val_prep, y_val))

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 606.3799 - mae: 22.5331 - val_loss: 608.0955 - val_mae: 22.9609
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 547.5219 - mae: 21.3059 - val_loss: 567.4774 - val_mae: 21.9365
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 482.5964 - mae: 20.0626 - val_loss: 534.4447 - val_mae: 20.9130
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 440.0460 - mae: 18.9318 - val_loss: 498.3188 - val_mae: 19.9040
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 413.9012 - mae: 18.0673 - val_loss: 462.3972 - val_mae: 19.5256
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 368.3705 - mae: 16.9594 - val_loss: 437.5225 - val_mae: 19.0142
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 352.3469 - mae: 16.4666 - val_loss: 413.9903 - val_mae: 18.4439
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 337.7796 - mae: 16.6085 - val_loss: 386.1531 - val_mae: 17.8399
Epoch 9/50
13/13 ━━━━━━

In [23]:
network.predict(X_test_prep)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[20.38338 ],
       [25.070776],
       [30.801296],
       [15.762549],
       [18.744112],
       [21.31631 ],
       [21.433159],
       [16.595266],
       [17.495636],
       [27.762104],
       [29.580847],
       [15.473222],
       [36.613445],
       [19.956245],
       [11.486778],
       [33.14249 ],
       [18.519495],
       [25.85533 ],
       [24.07431 ],
       [27.982384],
       [31.16378 ],
       [27.38081 ],
       [12.939869],
       [14.893938],
       [19.380386],
       [40.703945],
       [31.229296],
       [23.34746 ],
       [16.819124],
       [28.310642],
       [35.22857 ],
       [17.157434],
       [11.974957],
       [37.3786  ],
       [23.785194],
       [16.226864],
       [ 8.768951],
       [16.673027],
       [14.924321],
       [17.888462],
       [21.48945 ],
       [13.366656],
       [39.21309 ],
       [18.576746],
       [28.597527],
       [22.693178],
       [19.586185],
       [13.563029],
       [22.497644],
       [52.90706 ],
